In [ ]:
import tensorflow as tf
import tensorflow_docs as tfdocs
tf.keras.backend.clear_session()
print('Tensorflow version: %s' % str(tf.__version__))

import sys
import os
if sys.platform.startswith('darwin'):
    print('Enable multiple instances of OpenMP on macOS')
    os.environ['KMP_DUPLICATE_LIB_OK']='True'

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd
import pathlib
import shutil

In [ ]:
print('Prepare tensor board log path')
if 'logs' not in os.listdir():
    os.mkdir('logs')
logdir = pathlib.Path(os.getcwd() + '/logs')/"tensorboard_logs"
shutil.rmtree(logdir, ignore_errors=True)

In [ ]:
print('Download data file')
gz = tf.keras.utils.get_file('HIGGS.csv.gz', 'http://mlphysics.ics.uci.edu/data/higgs/HIGGS.csv.gz')

In [ ]:
print('Load data')

def pack_row(*row):
    label = row[0]
    features = tf.stack(row[1:],1)
    return features, label

FEATURE_CNT = 28
ds = tf.data.experimental.CsvDataset(gz,[float(),]*(FEATURE_CNT+1), compression_type="GZIP")
packed_ds = ds.batch(10000).map(pack_row).unbatch()
packed_ds

In [ ]:
print('Visualize data')
for features,label in packed_ds.batch(1000).take(1):
    print(features[0])
    plt.hist(features.numpy().flatten(), bins = 101)

In [ ]:
print('Split and batch data')

N_VALIDATION = int(1e3)
N_TRAIN = int(1e4)
BUFFER_SIZE = int(1e4)
BATCH_SIZE = 500
STEPS_PER_EPOCH = N_TRAIN//BATCH_SIZE

validate_ds = packed_ds.take(N_VALIDATION).cache()
train_ds = packed_ds.skip(N_VALIDATION).take(N_TRAIN).cache()

validate_ds = validate_ds.batch(BATCH_SIZE)
train_ds = train_ds.shuffle(BUFFER_SIZE).repeat().batch(BATCH_SIZE)

train_ds

In [ ]:
print('Prepare functions to train model')

lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
  0.001,
  decay_steps=STEPS_PER_EPOCH*1000,
  decay_rate=1,
  staircase=False) # schedule to reduce learning rate

def get_optimizer(): # optimization creation according to learning rate schedule
    return tf.keras.optimizers.Adam(lr_schedule)

class PrintDot(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        if epoch % 100 == 0: 
            print('')
        print('.', end='')

if 'cp' not in os.listdir():
    os.mkdir('cp') # prepare check point folder
        
def get_callbacks(name): # generate call back function list
    cp_dir = 'cp/' + name.split('/')[-1]
    cp_path = cp_dir + '/cp-{epoch:04d}.ckpt'
    if cp_dir not in os.listdir():
        os.mkdir(cp_dir)
    return [
        PrintDot(), # print dots during training
        tf.keras.callbacks.EarlyStopping(monitor='val_binary_crossentropy', patience=200), # early stop threshold
        tf.keras.callbacks.TensorBoard(logdir/name), # output log
        tf.keras.callbacks.ModelCheckpoint(filepath=cp_path, verbose=0, 
                                           save_weights_only=True, period=100) # checkpoint
    ]

def compile_and_fit(model, name, optimizer=None, max_epochs=10000): # compile and fit model
    if optimizer is None:
        optimizer = get_optimizer() # prepare optimizer
    model.compile(optimizer=optimizer,
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                  metrics=[tf.keras.losses.BinaryCrossentropy(from_logits=True, name='binary_crossentropy'),
                           'accuracy']) # compile model
    model.summary()
    history = model.fit(
        train_ds,
        steps_per_epoch = STEPS_PER_EPOCH,
        epochs=max_epochs,
        validation_data=validate_ds,
        callbacks=get_callbacks(name),
        verbose=0) # train model
    return history

In [ ]:
print('Prepare models to demonstrate overfitting')

models = [
    ('tiny', tf.keras.Sequential([
        tf.keras.layers.Dense(16, activation='elu', input_shape=(FEATURE_CNT,)),
        tf.keras.layers.Dense(1)
    ])),
    ('small', tf.keras.Sequential([
        tf.keras.layers.Dense(16, activation='elu', input_shape=(FEATURE_CNT,)),
        tf.keras.layers.Dense(16),
        tf.keras.layers.Dense(1)
    ])),
    ('medium', tf.keras.Sequential([
        tf.keras.layers.Dense(16, activation='elu', input_shape=(FEATURE_CNT,)),
        tf.keras.layers.Dense(16),
        tf.keras.layers.Dense(16),
        tf.keras.layers.Dense(1)
    ])),
    ('large', tf.keras.Sequential([
        tf.keras.layers.Dense(512, activation='elu', input_shape=(FEATURE_CNT,)),
        tf.keras.layers.Dense(512),
        tf.keras.layers.Dense(512),
        tf.keras.layers.Dense(512),
        tf.keras.layers.Dense(1)
    ]))
]

In [ ]:
print('Train different scale models')

save_path = 'saved_model' # prepare path to save model
if save_path not in os.listdir():
    os.mkdir(save_path)

size_histories = {}
for name, model in models:
    print('Train ', name, ' model...')
    size_histories[name] = compile_and_fit(model, 'sizes/'+name)
    model.save(save_path + '/' + name) # save model into a folder
    model.save(save_paht + '/' + name + '.h5') # save model into a hdf5 file

In [ ]:
print('Demonstrate histories')
plotter = tfdocs.plots.HistoryPlotter(metric = 'binary_crossentropy', smoothing_std=10)
plotter.plot(size_histories)
a = plt.xscale('log')
plt.xlim([5, max(plt.xlim())])
plt.ylim([0.5, 0.7])
plt.xlabel('Epochs [Log Scale]'')
plt.ylabel('Binary Crossentropy')

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

# Open an embedded TensorBoard viewer
%tensorboard --logdir {logdir}/sizes

In [ ]:
print('Try load model from saving')
print()

print('Original')
org_name, org_model = models[0]
org_model.summary()
